# Task 2: Recommendation Engine - Skeleton Notebook

This notebook provides a very basic example for the notebook you are expected to submit for Task 2 of the Final Project. The main purpose is that we can try different examples to get a better sense of your approach. Compared to Task 1 (Kaggle Competition), we don't have any objective means to evaluate the recommendations. 

Some general comments:
* You can import any data you need. This particularly includes your cleaned version of the Used Cars dataset; there's no need to show the data cleaning / preprocessing steps in this notebook.
* You can also import your code in form of external Python (.py) script. You're actually encouraged to do so to keep this notebook light and uncluttered.
* Please consider this notebook as an example and not to set specific requirements. As long there is a section where we can easily test your solution, it should be fine.

## Setting up the Notebook

In [1]:
import pandas as pd
import numpy as np

## Load the Data

For this example, we use a simplified version of the dataset with only 100 data sample, each with only 6 features

In [2]:
df_sample = pd.read_csv('data/sg-used-cars-final-simplified.csv')

df_sample.head()

,listing_id,make,power,engine_cap,mileage,price
0,1006025,bmw,135.0,1997.0,NaN,82300
1,994672,land rover,202.0,2993.0,25843.0,427900
2,921142,honda,95.6,1496.0,2000.0,109800
3,1008328,bmw,185.0,1998.0,57386.0,166600
4,1010661,NaN,96.0,1498.0,76000.0,59400


## Recommenders (Temporary)

In [3]:
# Normalization
def normalization(M):
    mj,mi=M.shape
    m_ary = M.values    
    m_ary = m_ary.T
    m_max = np.nanmax(M, axis=0)
    m_min = np.nanmin(M, axis=0)

    for i in range(mi):
        for j in range(mj):
            m_ary[i][j] = (m_ary[i][j] - m_min[i]) / (m_max[i] - m_min[i])
    
    return m_ary.T

In [4]:
# Matrix Factorization Init
def mf_init(M, k=100):
    # k is the size of the latent representation
    Z, W, H = None, None, None
    num_users, num_items = M.shape
    W = np.random.rand(num_users,k)
    H = np.random.rand(k,num_items)
    Z = np.argwhere(M>0)
    return Z, W, H

In [5]:
# Matrix Factorization Loss
def mf_loss(M, Z, W, H):
    loss = 0
    for i in Z:
        loss = loss + pow(M[i[0]][i[1]] - np.dot(W[i[0],:],H[:,i[1]]), 2)
    return loss

In [6]:
# Matrix Factorization Gradient Descent
def mf_gradientdescent(M, Z, W, H, k=100, learning_rate=0.0001, lambda_reg=0.1, num_iter=1000):
    for it in range(num_iter):
        for i in Z:
            eij = M[i[0]][i[1]] - np.dot(W[i[0],:],H[:,i[1]])
            for K in range(k):
                W[i[0]][K] = W[i[0]][K] + learning_rate * (2 * eij * H[K][i[1]] - 2 * lambda_reg * W[i[0]][K])
                H[K][i[1]] = H[K][i[1]] + learning_rate * (2 * eij * W[i[0]][K] - 2 * lambda_reg * H[K][i[1]])

        # Print loss every 10% of the iterations
        if(it % (num_iter/10) == 0):
            print('Loss: {:.5f} \t {:.0f}%'.format(mf_loss(M, Z, W, H), (it / (num_iter/100))))
    # Print final loss
    print('Loss: {:.5f} \t 100%'.format(mf_loss(M, Z, W, H)))
    return W, H

In [7]:
# Content-Based: Pairwise item similarity sim
def cos_sim(v1, v2):
    #v1 = v1 - np.mean(v1)
    #v2 = v2 - np.mean(v2)
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

## Computing the Top Recommendations

The method `get_top_recommendations()` shows an example of how to get the top recommendations for a given data sample (data sample = row in the dataframe of the dataset). The input is a row from the dataset and a list of optional input parameters which will depend on your approach; `k` is the number of returned recommendations seems useful, though.

The output should be a `pd.DataFrame` containing the recommendations. The output dataframe should have the same columns as the row + any additional columns you deem important (e.g., any score or tags that you might want to add to your recommendations).

In principle, the method `get_top_recommendations()` may be imported from a external Python (.py) script as well.

In [8]:
def get_top_recommendations(row, **kwargs) -> pd.DataFrame:
    
    #####################################################
    ## Initialize the required parameters
    
    # The number of recommendations seem recommended
    # Additional input parameters are up to you
    k = None
    df_init = df_sample.copy()
    similarity = list()
    row_id_sorted = list()
    #print("Original data:\n",df_init.head(),"\n")
    
    # Extract all **kwargs input parameters
    # and set the used paramaters (here: k)
    for key, value in kwargs.items():
        if key == 'k':
            k = value

    # Droped 'listing_id', 'make'
    df_sample_drp = df_init.drop(columns=['listing_id', 'make'])
    
    # Normalization
    df_sample_drp=normalization(df_sample_drp)
    
    # mf_gradientdescent
    np.random.seed(0) #debug
    Z, W, H = mf_init(df_sample_drp)
    W_new, H_new =mf_gradientdescent(df_sample_drp, Z, W, H, k=100, learning_rate=0.01, lambda_reg=0.1, num_iter=100)
    ratings_R = np.dot(W_new, H_new)
    df_P = pd.DataFrame(np.dot(W_new, H_new))    
    #print("mf_gradientdescent:\n",df_P.head(),"\n")

    #####################################################
    ## Compute your recommendations
    #
    # This is where your magic happens. Of course, you can call methods
    # defined in this notebook or in external Python (.py) scripts
    #

    # Pairwise item similarity sim
    row_mf = df_P.iloc[int(row.name)]
    for i in range(len(ratings_R)):
        similarity.append(cos_sim(row_mf, ratings_R[i]))

    # insert & sort by similarity 
    df_result_tmp=df_init
    df_result_tmp.insert(len(df_result_tmp.columns), 'similarity', similarity)
    df_result_tmp = df_result_tmp.sort_values('similarity', ascending=False)
    #print("Similarity Sorted with original data:\n",df_result_tmp.head(),"\n")
    
    # get recommend row_id
    for i in range(1,k+1):
        row_id_sorted.append(df_result_tmp.index[i])
    
    # Here, we just return the input row k times
    # Ideally, you recommendations will be much better
    df_result = pd.DataFrame(df_init.iloc[row_id_sorted], index=None)
        
    # Return the dataset with the k recommendations
    return df_result


## Testing the Recommendation Engine

This will be the main part of your notebook to allow for testing your solutions. Most basically, for a given listing (defined by the row id in your input dataframe), we would like to see the recommendations you make. So however you set up your notebook, it should have at least a comparable section that will allow us to run your solution for different inputs.

### Pick a Sample Listing as Input

In [9]:
# Pick a row id of choice
row_id = 10
#row_id = 20
#row_id = 30
#row_id = 40
#row_id = 50

# Get the row from the dataframe (an valid row ids will throw an error)
row = df_sample.iloc[row_id]

# Just for printing it nicely, we create a new dataframe from this single row
pd.DataFrame([row])

,listing_id,make,power,engine_cap,mileage,price
10,1020216,honda,73.0,1317.0,22703.0,78000


## Compute and Display the recommendations

Since the method `get_top_recommendations()` returns a `pd.DataFrame`, it's easy to display the result.

In [10]:
k = 3

df_recommendations = get_top_recommendations(row, k=k)

df_recommendations.head(k)

Loss: 1585.32569 	 0%
Loss: 11.45951 	 10%
Loss: 9.33004 	 20%
Loss: 9.06115 	 30%
Loss: 8.85863 	 40%
Loss: 8.68558 	 50%
Loss: 8.53675 	 60%
Loss: 8.40977 	 70%
Loss: 8.30075 	 80%
Loss: 8.20463 	 90%
Loss: 8.12490 	 100%


,listing_id,make,power,engine_cap,mileage,price,similarity
70,988825,nissan,85.0,1598.0,36868.0,78000,0.999296
85,1022601,honda,73.0,1339.0,NaN,51500,0.998969
59,1010728,honda,73.0,1318.0,29000.0,76400,0.998887
